### Create Assistant

In [8]:
from openai import OpenAI
 
client = OpenAI()
 
assistant = client.beta.assistants.create(
  name="Financial Analyst Assistant-1",
  instructions="You are an expert. Use you knowledge base to answer questions",
  model="gpt-4o",
  tools=[{"type": "file_search"}],
)

### Parse Data and Upload to Vector Store DB

In [9]:
# Create a vector store caled "Financial Statements"
vector_store = client.beta.vector_stores.create(name="cinamon_rp")
 
# Ready the files for upload to OpenAI
file_paths = ["data/Cinnamon.pdf"]
file_streams = [open(path, "rb") for path in file_paths]
 
# Use the upload and poll SDK helper to upload the files, add them to the vector store,
# and poll the status of the file batch for completion.
file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
  vector_store_id=vector_store.id, files=file_streams
)
 
# You can print the status and the file counts of the batch to see the result of this operation.
print(file_batch.status)
print(file_batch.file_counts)

completed
FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)


### Update  Assistant With Vecotr Store

In [10]:
assistant = client.beta.assistants.update(
  assistant_id=assistant.id,
  tool_resources={"file_search": {"vector_store_ids": [vector_store.id]}},
)

### Create Thread

In [11]:
# Upload the user provided file to OpenAI
message_file = client.files.create(
  file=open("data/aapl-10k.pdf", "rb"), purpose="assistants"
)
 
# Create a thread and attach the file to the message
thread = client.beta.threads.create(
  messages=[
    {
      "role": "user",
      "content": "How many shares of AAPL were outstanding at the end of of October 2023?",
      # Attach the new file to the message.
      "attachments": [
        { "file_id": message_file.id, "tools": [{"type": "file_search"}] }
      ],
    }
  ]
)
 
# The thread now has a vector store with that file in its tool resources.
print(thread.tool_resources.file_search)

ToolResourcesFileSearch(vector_store_ids=['vs_qHSEI6MDOQahu9avIEaBwwHj'])


In [12]:
vector_store.id

'vs_X09wu6TDk1HJsPyWvMHBIOI3'

### Run The Assistant

In [13]:
# Use the create and poll SDK helper to create a run and poll the status of
# the run until it's in a terminal state.

run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id, assistant_id=assistant.id
)

messages = list(client.beta.threads.messages.list(thread_id=thread.id, run_id=run.id))

message_content = messages[0].content[0].text
annotations = message_content.annotations
citations = []
for index, annotation in enumerate(annotations):
    message_content.value = message_content.value.replace(annotation.text, f"[{index}]")
    if file_citation := getattr(annotation, "file_citation", None):
        cited_file = client.files.retrieve(file_citation.file_id)
        citations.append(f"[{index}] {cited_file.filename}")

print(message_content.value)
print("\n".join(citations))

At the end of October 2023, Apple Inc. had approximately 15,550,061 thousand shares of common stock outstanding[0].
[0] aapl-10k.pdf
